In [59]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
import lightgbm as lgb
from catboost import CatBoostRegressor

In [60]:
datafile_train="/content/Hazard_train.csv"
datafile_test="/content/Hazard_test_share.csv"
train_data=pd.read_csv(datafile_train)
test_data=pd.read_csv(datafile_test)

In [36]:
print(train_data.info())
print(test_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40799 entries, 0 to 40798
Data columns (total 34 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Id      40799 non-null  int64 
 1   Hazard  40799 non-null  int64 
 2   T1_V1   40799 non-null  int64 
 3   T1_V2   40799 non-null  int64 
 4   T1_V3   40799 non-null  int64 
 5   T1_V4   40799 non-null  object
 6   T1_V5   40799 non-null  object
 7   T1_V6   40799 non-null  object
 8   T1_V7   40799 non-null  object
 9   T1_V8   40799 non-null  object
 10  T1_V9   40799 non-null  object
 11  T1_V10  40799 non-null  int64 
 12  T1_V11  40799 non-null  object
 13  T1_V12  40799 non-null  object
 14  T1_V13  40799 non-null  int64 
 15  T1_V14  40799 non-null  int64 
 16  T1_V15  40799 non-null  object
 17  T1_V16  40799 non-null  object
 18  T1_V17  40799 non-null  object
 19  T2_V1   40799 non-null  int64 
 20  T2_V2   40799 non-null  int64 
 21  T2_V3   40799 non-null  object
 22  T2_V4   40799 non-null

In [61]:
X = train_data.drop(columns=['Hazard'])
y = train_data['Hazard']

In [62]:
data_combined = pd.concat([X, test_data], axis=0, ignore_index=True)

In [63]:
categorical_columns = data_combined.select_dtypes(include=['object']).columns
for col in categorical_columns:
    le = LabelEncoder()
    data_combined[col] = le.fit_transform(data_combined[col].astype(str))

In [64]:
imputer = SimpleImputer(strategy='mean')
data_combined = pd.DataFrame(imputer.fit_transform(data_combined), columns=data_combined.columns)

In [65]:
X = data_combined.iloc[:len(X), :]
X_test = data_combined.iloc[len(X):, :]

In [66]:
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

In [67]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [68]:
def objective_lightgbm(trial):
    params = {
        'objective': 'regression',
        'metric': 'mae',
        'boosting_type': 'gbdt',
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
    }
    model = lgb.LGBMRegressor(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='mae',
        callbacks=[
            lgb.early_stopping(stopping_rounds=20),
            lgb.log_evaluation(period=10)
        ]
    )
    y_pred = model.predict(X_val)
    return mean_absolute_error(y_val, y_pred)

In [69]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_lightgbm, n_trials=50)
best_params_lgb = study.best_params
print("Best LightGBM params:", best_params_lgb)

[I 2025-01-04 07:29:47,227] A new study created in memory with name: no-name-3a7156eb-6c30-45da-b64f-86ff391d0bc5


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.018982 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 677
[LightGBM] [Info] Number of data points in the train set: 32639, number of used features: 33
[LightGBM] [Info] Start training from score 4.002911
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

[I 2025-01-04 07:29:49,972] Trial 0 finished with value: 2.7211545405799384 and parameters: {'max_depth': 3, 'num_leaves': 34, 'learning_rate': 0.04084137180443763, 'n_estimators': 493, 'subsample': 0.9070577043477117, 'colsample_bytree': 0.7406909981355089}. Best is trial 0 with value: 2.7211545405799384.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005389 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 677
[LightGBM] [Info] Number of data points in the train set: 32639, number of used features: 33
[LightGBM] [Info] Start training from score 4.002911
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM

[I 2025-01-04 07:29:51,334] Trial 1 finished with value: 2.721026599180957 and parameters: {'max_depth': 5, 'num_leaves': 72, 'learning_rate': 0.04344758235791478, 'n_estimators': 373, 'subsample': 0.6443793788385184, 'colsample_bytree': 0.9478209799511436}. Best is trial 1 with value: 2.721026599180957.


...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011946 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 677
[LightGBM] [Info] Number of data points in the train set: 32639, number of used features: 33
[LightGBM] [Info] Start training from score 4.002911
Training until validation scores don't improve for 20 rounds
[10]	valid_0's l1: 2.80329
[20]	valid_0's l1: 2.77962
[30]	valid_0's l1: 2.76275
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[40]	valid_0's l1: 2.75163
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[50]	valid_0's l1: 2.74219
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[60]	valid_0's l1: 2.73611
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

[I 2025-01-04 07:31:05,821] Trial 48 finished with value: 2.716335245353908 and parameters: {'max_depth': 7, 'num_leaves': 71, 'learning_rate': 0.048558004861340265, 'n_estimators': 317, 'subsample': 0.7728290544043173, 'colsample_bytree': 0.6021985434458239}. Best is trial 18 with value: 2.714530685831164.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010659 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 677
[LightGBM] [Info] Number of data points in the train set: 32639, number of used features: 33
[LightGBM] [Info] Start training from score 4.002911
Training until validation scores don't improve for 20 rounds
[10]	valid_0's l1: 2.80348
[20]	valid_0's l1: 2.78016
[30]	valid_0's l1: 2.7629
[40]	valid_0's l1: 2.75159
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[50]	valid_0's l1: 2.74003
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[60]	valid_0's l1: 2.73391
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

[I 2025-01-04 07:31:07,335] Trial 49 finished with value: 2.717747545798562 and parameters: {'max_depth': 7, 'num_leaves': 70, 'learning_rate': 0.04866773336605622, 'n_estimators': 308, 'subsample': 0.7712439857226142, 'colsample_bytree': 0.6017869266737679}. Best is trial 18 with value: 2.714530685831164.


Best LightGBM params: {'max_depth': 7, 'num_leaves': 60, 'learning_rate': 0.0687197112521769, 'n_estimators': 297, 'subsample': 0.8682851174485697, 'colsample_bytree': 0.6526674363761874}


In [70]:
final_lgb_model = lgb.LGBMRegressor(**best_params_lgb)
final_lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='mae',
    callbacks=[
        lgb.early_stopping(stopping_rounds=20),
        lgb.log_evaluation(period=10)
    ]
)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012007 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 677
[LightGBM] [Info] Number of data points in the train set: 32639, number of used features: 33
[LightGBM] [Info] Start training from score 4.002911
Training until validation scores don't improve for 20 rounds
[10]	valid_0's l1: 2.79299	valid_0's l2: 14.8958
[20]	valid_0's l1: 2.7684	valid_0's l2: 14.5267
[30]	valid_0's l1: 2.75113	valid_0's l2: 14.3267
[40]	valid_0's l1: 2.73746	valid_0's l2: 14.1988
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[50]	valid_0's l1: 2.73068	valid_0's l2: 14.1554
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

LGBMRegressor(colsample_bytree=0.6526674363761874,
              learning_rate=0.0687197112521769, max_depth=7, n_estimators=297,
              num_leaves=60, subsample=0.8682851174485697)

In [71]:
y_pred_lgb = final_lgb_model.predict(X_val)
mae_lgb = mean_absolute_error(y_val, y_pred_lgb)
score_lgb = 1 - (mae_lgb / 5.4)
print(f"LightGBM Validation MAE: {mae_lgb}, Score: {score_lgb}")


LightGBM Validation MAE: 2.7168946734168653, Score: 0.49687135677465466


In [72]:
cat_model = CatBoostRegressor(iterations=500, depth=6, learning_rate=0.05, loss_function='MAE', verbose=0)
cat_model.fit(X_train, y_train)
y_pred_cat = cat_model.predict(X_val)
mae_cat = mean_absolute_error(y_val, y_pred_cat)
score_cat = 1 - (mae_cat / 5.4)
print(f"CatBoost Validation MAE: {mae_cat}, Score: {score_cat}")


CatBoost Validation MAE: 2.5852540755461844, Score: 0.5212492452692252


In [73]:
if score_lgb > score_cat:
    print("LightGBM selected.")
    best_model = final_lgb_model
else:
    print("CatBoost selected.")
    best_model = cat_model


CatBoost selected.


In [74]:
test_predictions = best_model.predict(X_test)

In [ ]:
submission = pd.DataFrame({
    'Hazard': test_predictions
})

# Save the submission to a CSV file
submission.to_csv("output.csv", index=False)
print("Submission file created: submission.csv")